In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np

import klib as kl


In [4]:
final_cols = None

### Web Traffic Cleanup

In [5]:

products_data = pd.read_excel('./data/Account Products.xlsx', sheet_name='web_traffic_accounts')
products_data.Date = pd.to_datetime(products_data.Date)
products_data.set_index('Date', inplace=True)

final_cols = products_data.columns

imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = imputer.fit_transform(products_data)
imputed_data = pd.DataFrame(imputed_data, columns=products_data.columns, index=products_data.index)
products_data = imputed_data.copy()

# get bank name from columns 
products_data.columns = products_data.columns.str.split('.').str[0]


numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_data = products_data.select_dtypes(include=numerics)
num_cols_target = numeric_data.columns

products_data.head()

,abanca,n26,bancsabadell,ing,bbva,revolut,openbank,myinvestor,n26,bankinter,ing,evobanco,bancosantander
Date,,,,,,,,,,,,,
2021-12-01,15202.983835,6664.368911,20611.877467,24449.393654,45449.486787,1343.418328,15254.582228,5319.608623,15239.190071,8733.560835,24448.706786,5055.542564,22325.885971
2022-01-01,42183.913207,5310.177360,28890.415605,24463.625737,38306.464983,1531.016801,17359.852958,6577.851993,11468.598706,11410.593443,24463.492621,4539.012047,28331.245044
2022-02-01,34498.153115,5547.588971,24924.681184,24456.839455,40978.325821,1304.591257,16944.054696,4031.716960,12928.637519,8891.661053,24456.441669,6049.457905,23798.819170
2022-03-01,34546.319021,4900.271300,23968.596015,24455.725835,43970.545627,1105.180575,16387.002853,4941.009311,12554.646465,7446.925144,24455.281024,9826.193608,24808.561729
2022-04-01,40508.288420,4909.581023,27150.311205,24461.184509,40782.451956,1020.968713,17414.866042,5774.734455,11887.397875,10285.861726,24460.962770,5028.236254,24739.758729


### Bank Information Cleanup

In [6]:



bank_comps = pd.read_excel('data/Account Products.xlsx', sheet_name='bank_comparison_metrics', header=0)
bank_comps['has_stores'] = bank_comps['number of branches'] > 0

bank_comps.head()

,Notes,website,number of employee,number of customers,assets under management,number of branches,age of bank,references,Unnamed: 8,Unnamed: 9,has_stores
0,NaN,abanca.com,5946,NaN,7.214800e+10,690,2011,https://www.abancacorporacionbancaria.com/file...,NaN,NaN,True
1,NaN,bancosantander.es,212764,NaN,1.117000e+12,8518,1857,https://www.santander.com/en/about-us/key-fact...,https://www.statista.com/statistics/417009/ban...,NaN,True
2,NaN,bancsabadell.com,19316,NaN,2.530000e+11,1594,1881,https://www.globaldata.com/company-profile/ban...,https://www.bancsabadell.com/bsnacional/en/bra...,NaN,True
3,NaN,bankinter.com,6138,NaN,8.330000e+10,523,1965,https://www.macrotrends.net/stocks/charts/BKNI...,https://www.bankinter.com/webcorporativa/amp/e...,https://www.bankinter.com/www/webcorp/swf/memo...,True
4,NaN,bbva.es,121486,NaN,7.750000e+11,1800,1857,https://www.globaldata.com/company-profile/ban...,https://www.bbva.es/en/general/buscador-oficin...,NaN,True


### Cross Visitation Classification

In [7]:
cross_visitation_data = pd.read_excel('data/Account Products.xlsx', sheet_name='cross_visitation', header=1)

cross_visitation_data = kl.data_cleaning(cross_visitation_data)

brick_and_mortar_banks = list(bank_comps.loc[bank_comps.has_stores == True, 'website'])
neo_banks = [x for x in cross_visitation_data.columns if x not in brick_and_mortar_banks]
neo_banks.remove('date')


cross_visitation_data.head()

Shape of cleaned data: (25, 16) - Remaining NAs: 25


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.0 MB (-nan%)



/Users/andrewbennett/Library/Python/3.9/lib/python/site-packages/klib/utils.py:142: RuntimeWarning: invalid value encountered in scalar divide
  mem_perc = round(100 * mem_change / data_mem, 2)


,date,abanca_com,bancosantander_es,bancsabadell_com,bankinter_com,bbva_es,caixabank_es,evobanco_com,imagin_com,ing_es,kutxabank_es,myinvestor_es,n26_com,openbank_es,pibank_es,revolut_com
0,2021-12-01,0.077994,0.116923,0.070229,0.105769,0.196307,NaN,0.079812,0.059727,0.120856,NaN,0.068323,0.067204,0.128280,NaN,0.071006
1,2022-01-01,0.062500,0.107472,0.084605,0.094186,0.170377,NaN,0.055328,0.061053,0.121270,NaN,0.067114,0.078723,0.127622,0.059259,0.052795
2,2022-02-01,0.052288,0.119243,0.059621,0.083083,0.192536,NaN,0.067478,0.065734,0.098605,0.048193,0.049738,0.055477,0.117399,0.057143,0.049020
3,2022-03-01,0.061503,0.112708,0.067651,0.082576,0.174929,NaN,0.075472,0.055402,0.120556,NaN,0.059060,0.056548,0.107581,0.034884,0.051075
4,2022-04-01,0.050222,0.128592,0.064007,0.102992,0.181480,NaN,0.072148,0.058762,0.114540,NaN,0.050104,0.052254,0.104252,0.037618,0.045169


In [ ]:
# get numerica columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_data = cross_visitation_data.select_dtypes(include=numerics)
num_cols = numeric_data.columns


### Account Ranking Preprocessing

In [12]:

file_path = './data/Account Products.xlsx'
df = pd.read_excel(file_path, sheet_name='ranking_position', skiprows=7)


In [13]:
df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')

# Rename the columns for clarity:
df.columns = ['Category', 'Bank Name'] + list(df.columns[2:])

transformed_df = pd.melt(df, id_vars=['Category', 'Bank Name'], var_name='Date', value_name='Rank')
transformed_df['Date'] = pd.to_datetime(transformed_df['Date'])

/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_40810/1715715144.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Unnamed: 0'] = df['Unnamed: 0'].fillna(method='ffill')


In [14]:
rank_df = transformed_df

rank_df.head()

,Category,Bank Name,Date,Rank
0,Best Comission-Free Accounts (Helmycash),Cuenta Online sin comisiones BBVA,2021-12-01,1.0
1,Best Comission-Free Accounts (Helmycash),Cuenta Nómina ING,2021-12-01,2.0
2,Best Comission-Free Accounts (Helmycash),Cuenta Nómina hasta 5% TAE Bankinter,2021-12-01,3.0
3,Best Comission-Free Accounts (Helmycash),Cuenta Clara ABANCA,2021-12-01,4.0
4,Best Comission-Free Accounts (Helmycash),Cuenta Online SIN Liberbank,2021-12-01,5.0


### Bank Incentives Clean Up

In [15]:
data = pd.read_csv('data/site_changes.csv')
# rename tags

data.tag = data.tag.replace({'apr increase':'increase apr', 'incentive increase':'increase incentive', 'decrease incentive':'incentive decrease',  'decrease incentive ':'incentive decrease'})

# create a date column from month and year and add day as 1
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))

apr_increase_dates = data[data.tag == 'increase apr']
incentive_increase_dates = data[data.tag == 'increase incentive']

incentive_increase_dates.head()

,month,year,incentive,apr,notes,bank,notes.1,tag,site changes,date
1,6,2021,150€,0%,800-1200 deposit,abanca,NaN,increase incentive,NaN,2021-06-01
5,7,2023,300€,2%,NaN,abanca,NaN,increase incentive,NaN,2023-07-01
9,9,2022,0,1%,por el primer año,banc sabadell,NaN,increase incentive,NaN,2022-09-01
13,4,2023,250,2%,por el primer año,banc sabadell,250â¬ por domiciliar tu nÃ³mina,increase incentive,NaN,2023-04-01
18,7,2023,0,5%,añadir un balance maximo a 10000 por un total ...,bankinter,NaN,increase incentive,NaN,2023-07-01


In [16]:
apr_increase_dates.head()

,month,year,incentive,apr,notes,bank,notes.1,tag,site changes,date
3,4,2022,300€,0%,if over 1200 deposit,abanca,NaN,increase apr,NaN,2022-04-01
6,12,2023,300€,2%,NaN,abanca,NaN,increase apr,NaN,2023-12-01
12,1,2023,175,2%,por el primer año,banc sabadell,175€ si domicilias tu nomina,increase apr,NaN,2023-01-01
32,11,2022,NaN,0.06%,mejor cambia de el sitio,evobanco,NaN,increase apr,NaN,2022-11-01
33,4,2023,NaN,2.00%,aumenta el incentivo,evobanco,NaN,increase apr,NaN,2023-04-01


### News Release Clean Up

### Data Consolidation by Bank